<h1><center>CMPE 462 - Project 2<br>Implementing an SVM Classifier<br>Due: May 18, 2020, 23:59</center></h1>

* **Student ID1:** 2015400003
* **Student ID2:**
* **Student ID3:**

## Overview

In this project, you are going to implement SVM. For this purpose, a data set (data.mat) is given to you. You can load the mat dataset into Python using the function `loadmat` in `Scipy.io`. When you load the data, you will obtain a dictionary object, where `X` stores the data matrix and `Y` stores the labels. You can use the first 150 samples for training and the rest for testing. In this project, you will use the software package [`LIBSVM`](http://www.csie.ntu.edu.tw/~cjlin/libsvm/) to implement SVM. Note that `LIBSVM` has a [`Python interface`](https://github.com/cjlin1/libsvm/tree/master/python), so you can call the SVM functions in Python. 

In [1]:
# Install a pip package in the current Jupyter kernel
#import sys
#!{sys.executable} -m pip install libsvm

In [2]:
from scipy.io import loadmat
import numpy as np
from libsvm.svmutil import *
import io
from contextlib import redirect_stdout

## Task 1 - 30 pts

Train a hard margin linear SVM and report both train and test classification accuracy.

In [3]:
mat_data = loadmat('data.mat')

x, y = mat_data['X'], np.squeeze(mat_data['Y'])

train_x, test_x = x[:150], x[150:]
train_y, test_y = y[:150], y[150:]

SVM_PROBLEM = svm_problem(train_y, train_x)

# Setting c to a large value here (99..9) to train hard margin SVM.
hard_svm_parameter = svm_parameter('-c 999999999')
hard_svm_model = svm_train(SVM_PROBLEM, hard_svm_parameter)

train_res=svm_predict(train_y, train_x, hard_svm_model)
test_res=svm_predict(test_y, test_x, hard_svm_model)

# Accuracy:
# print(train_res[1][0])
# print(test_res[1][0])

Accuracy = 100% (150/150) (classification)
Accuracy = 76.6667% (92/120) (classification)


## Task 2 - 40 pts

Train soft margin SVM for different values of the parameter $C$, and with different kernel functions. Systematically report your results. For instance, report the performances of different kernels for a fixed $C$, then report the performance for different $C$ values for a fixed kernel, and so on.

In [4]:
kernels = [0,1,2,3]

# Realistic c values needed here. Current value list contains powers of 2. 
c_vals = [2**x for x in range(5)] 

results = {}
for k in kernels:
    k_res = {}
    for c in c_vals:
        soft_svm_parameter = svm_parameter('-c {} -t {}'.format(c,k))
        soft_svm_model = svm_train(SVM_PROBLEM, soft_svm_parameter)
        trap = io.StringIO()
        with redirect_stdout(trap):
            # svm_predict prints accuracy to stdout. Surpressing here.
            train_res=svm_predict(train_y, train_x, soft_svm_model)
            test_res=svm_predict(test_y, test_x, soft_svm_model)
        k_res[c] = (train_res[1][0], test_res[1][0], soft_svm_model) 
    results[k] = k_res        

In [5]:
for k,res in results.items():
    print("Kernel:",k)
    for c,r in res.items():
        print("C:", c, "|| Train:", round(r[0], 1), ", Test:", round(r[1], 1), ", Vector count:", len(r[2].get_sv_indices()))
    print("==========")    
        

Kernel: 0
C: 1 || Train: 86.7 , Test: 85.0 , Vector count: 58
C: 2 || Train: 88.0 , Test: 83.3 , Vector count: 56
C: 4 || Train: 88.7 , Test: 80.8 , Vector count: 54
C: 8 || Train: 88.7 , Test: 81.7 , Vector count: 52
C: 16 || Train: 88.7 , Test: 81.7 , Vector count: 50
Kernel: 1
C: 1 || Train: 86.0 , Test: 82.5 , Vector count: 118
C: 2 || Train: 88.7 , Test: 83.3 , Vector count: 100
C: 4 || Train: 90.0 , Test: 81.7 , Vector count: 90
C: 8 || Train: 92.7 , Test: 82.5 , Vector count: 87
C: 16 || Train: 95.3 , Test: 81.7 , Vector count: 84
Kernel: 2
C: 1 || Train: 86.7 , Test: 84.2 , Vector count: 83
C: 2 || Train: 88.7 , Test: 82.5 , Vector count: 76
C: 4 || Train: 90.7 , Test: 83.3 , Vector count: 76
C: 8 || Train: 94.7 , Test: 78.3 , Vector count: 73
C: 16 || Train: 96.0 , Test: 77.5 , Vector count: 73
Kernel: 3
C: 1 || Train: 82.7 , Test: 84.2 , Vector count: 79
C: 2 || Train: 84.7 , Test: 85.0 , Vector count: 69
C: 4 || Train: 82.7 , Test: 82.5 , Vector count: 61
C: 8 || Train: 82.0

## Task 3 - 15 pts

Please report how the number of support vectors changes as the value of $C$ increases (while all other parameters remain the same). Discuss whether your observations match the theory.

In [6]:
fixed_kernel_param = 3 # arbitrarily chosen.

for c,r in results[fixed_kernel_param].items():
    print("C:", c, "||", len(r[2].get_sv_indices()))

C: 1 || 79
C: 2 || 69
C: 4 || 61
C: 8 || 57
C: 16 || 53


## Task 4 - 15 pts

Please investigate the changes in the hyperplane when you remove one of the support vectors, vs., one data point that is not a support vector.

In [7]:
# https://stackoverflow.com/questions/10131385/matlab-libsvm-how-to-find-the-w-coefficients

### Bonus Task - 10 pts

Use Python and [CVXOPT](http://cvxopt.org) QP solver to implement the hard margin SVM. 